# Library


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
import os

In [3]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# Fold

In [4]:
df1_fold = 'D:/data/big2/BD1/df'

# server

In [5]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSE')

In [6]:
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

# Load
- 등록정보&제원정보 -> result 4등급(연료 전체)
- 정기검사&정밀검사

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [7]:
# 8.6s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
}
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1906876 entries, 0 to 1906875
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   VIN                object 
 1   BSPL_STDG_CD       int64  
 2   EXHST_GAS_GRD_CD   object 
 3   EXHST_GAS_CERT_NO  object 
 4   VHCL_ERSR_YN       object 
 5   MANP_MNG_NO        object 
 6   YRIDNW             int64  
 7   VHCTY_CD           object 
 8   PURPS_CD2          object 
 9   FRST_REG_YMD       float64
 10  VHCL_FBCTN_YMD     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 160.0+ MB


In [8]:
carr = car.rename(columns=car_ch_col)
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [9]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}
src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566615 entries, 0 to 566614
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   MANP_MNG_NO   566615 non-null  object
 1   FUEL_CD       545937 non-null  object
 2   VHCTY_TY_CD2  553038 non-null  object
 3   MNFCTR_NM     559794 non-null  object
 4   VHCNM         566544 non-null  object
 5   VHCL_FRM      536460 non-null  object
 6   EGIN_TY       517013 non-null  object
dtypes: object(7)
memory usage: 30.3+ MB


In [10]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 정기검사(STD_TB_JGT_HIS)

In [11]:
# 3m 34.9s
jgt = wd.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS;")
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_JGMT':'검사판정', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_DRVNG_DSTNC':'주행거리',
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}
jgt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65903838 entries, 0 to 65903837
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   VIN                     object 
 1   FDRM_INSP_INSP_MTHD_CD  object 
 2   FDRM_INSP_KND_CD        object 
 3   FDRM_INSP_JGMT          object 
 4   FDRM_NLOD_SMO_MSTVL1    float64
 5   FDRM_NLOD_SMO_MSTVL2    float64
 6   FDRM_NLOD_SMO_MSTVL3    float64
 7   FDRM_NLOD_SMO_JT_YN1    object 
 8   FDRM_INSP_YMD           int64  
 9   FDRM_DRVNG_DSTNC        float64
 10  FDRM_NLOD_SMO_PRMVL1    float64
dtypes: float64(5), int64(1), object(5)
memory usage: 5.4+ GB


In [12]:
jgtr = jgt.rename(columns=jgt_ch_col)
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 정밀검사(STD_TB_EET_HIS_ME)

In [13]:
# 6m 36.1s
eet = wd.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PREC_NLOD_SMO_PRMVL1 FROM STD_TB_EET_HIS_ME;")
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}
eet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102191938 entries, 0 to 102191937
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VIN                    object 
 1   PRCINSP_MSRMT_MTHD_CD  object 
 2   PRCINSP_KND_CD         object 
 3   PRCINSP_JGMT           object 
 4   PREC_NLOD_SMO_MSTVL1   float64
 5   PREC_NLOD_SMO_MSTVL2   float64
 6   PREC_NLOD_SMO_MSTVL3   float64
 7   PREC_NLOD_SMO_JT_YN1   object 
 8   PRCINSP_YMD            int64  
 9   PRCINSP_DRVNG_DSTNC    float64
 10  PREC_NLOD_SMO_PRMVL1   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 8.4+ GB


In [14]:
eetr = eet.rename(columns=eet_ch_col)
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [15]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47916 entries, 0 to 47915
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       47916 non-null  int64 
 1   STDG_CTPV_NM  47916 non-null  object
 2   STDG_SGG_NM   47892 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [16]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## RH에서 제공한 법정동코드

In [17]:
# 1.3s
rh = we.export_to_pandas("SELECT DONG_CODE, CTPRVN_NM, SIGNGU_NM FROM STD_BD_TB_MAPDATA;")
rh = rh.rename(columns={'DONG_CODE':'법정동코드_rh', 'CTPRVN_NM':'시도', 'SIGNGU_NM':'시군구'})
rh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   법정동코드_rh  3799 non-null   int64 
 1   시도        3799 non-null   object
 2   시군구       3757 non-null   object
dtypes: int64(1), object(2)
memory usage: 89.2+ KB


## 4등급 result(for DPF유무)

In [18]:
# 3.3s
rs = we.export_to_pandas("SELECT 차대번호, DPF유무_수정 FROM STD_BD_GRD4_RESULT;")
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   차대번호      1502235 non-null  object
 1   DPF유무_수정  1153813 non-null  object
dtypes: object(2)
memory usage: 22.9+ MB


# 전처리

## 중복 차대번호 제거

In [19]:
carr.shape, len(carr['차대번호'].unique())

((1906876, 11), 1906823)

In [20]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1906823, 11)

## 배출가스등급 코드 변환

In [21]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['4'], dtype=object)

## 차종 코드 변환

In [22]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['승용', '화물', '승합', '특수'], dtype=object)

## 용도 코드 변환

In [23]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [24]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1390280, 11)

## 연료 코드 변환

In [25]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['휘발유', '경유', '기타연료', 'LPG(액화석유가스)', nan, '휘발유 하이브리드',
       'CNG(압축천연가스)', '알코올', '등유', '전기', '태양열', '경유 하이브리드', 'LPG 하이브리드',
       '수소', 'LNG(액화천연가스)', 'CNG 하이브리드'], dtype=object)

## 차종유형 코드 변환

In [26]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '중형', '소형', '경형', nan], dtype=object)

## 등록&제원 병합

In [27]:
# 0.7s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1390280, 17)

In [28]:
cs.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,제작일자,연료,차종유형,제작사명,차명,자동차형식,엔진형식
0,KMFZSX7JABU684137,4374025000,4,NaN,N,A0810006209153110,2011,화물,개인용,20220923.00,20101022.00,경유,소형,현대자동차(주),포터Ⅱ(PORTERⅡ),HRTJA-DX-B,D4CB
1,KNCSE06428K344169,4374025000,4,NaN,N,A0110003113533107,2008,화물,개인용,20220923.00,20080721.00,경유,소형,기아자동차(주),봉고Ⅲ 1톤,SEE42J-A,J3
2,KMCGK17HP9C112030,4518010400,4,7MY-HD-24-28,N,A0810005008143209,2009,화물,개인용,20220407.00,20090316.00,경유,중형,현대자동차(주),마이티군물자수송트럭,HD25-9MT0-LHFbF,D4GA
3,KNCSE01429K388374,2771026522,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210909.00,20090312.00,경유,소형,기아자동차(주),봉고Ⅲ 1톤,SET42K-A,J3
4,KNFSE0A439K387582,3020011900,4,7MY-KM-14-65,N,A0110003116073108,2009,화물,개인용,20210902.00,20090320.00,경유,소형,기아자동차(주),봉고Ⅲ 냉동차,SEL43K-RTSS,J3


## 정기&정밀 병합

In [29]:
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [30]:
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [31]:
jgtr['정기정밀'] = '정기'
eetr['정기정밀'] = '정밀'

In [32]:
# 39.6s
ins = pd.concat([jgtr, eetr], ignore_index=True)
ins.shape

(168095776, 12)

### 최근 검사만 활용

In [33]:
# 4m 14.3s
insm = ins.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
insm.shape

(34641431, 12)

## 검사종류 코드 변환

In [34]:
# 27.5s
knd_dict = {
    'A21P01':'정밀검사', 
    'A1904':'정기검사', 
    'A21T06':'종합검사(경과)', 
    'A21T05':'종합검사', 
    'A21S01':'특정검사', 
    'A1902':'재검사(2부재검사)',
    'A21R02':'2부재검사(종합)', 
    'A21P02':'정밀검사(경과)', 
    'A1903':'정기(경과)검사', 
    'A21S04':'특정재검사(배출)', 
    'A21E01':'배출재검사', 
    'A21R01':'1부재검사(종합)',
    'A21C02':'구조변경검사', 
    'A21002':'관능재검사(임시)', 
    'A21S03':'특정재검사(관능)', 
    'A21S02':'특정검사(경과)', 
    'A1901':'재검사(1부재검사)', 
    'A21T02':'종합(정밀)',
    'A21E02':'배출재검사(구변)', 
    'A21T01':'종합(경과:정밀)', 
    'A21T04':'종합(정밀) 2부재검사', 
    'A21001':'관능재검사', 
    'A21T03':'종합(정밀) 1부재검사', 
    'A21TMP':'임시검사', 
}
insm['검사종류'] = insm['검사종류'].replace(knd_dict)
insm['검사종류'].unique()

array(['종합검사', '1부재검사(종합)', '종합검사(경과)', '2부재검사(종합)', '정기검사', '재검사(1부재검사)',
       '재검사(2부재검사)', '정기(경과)검사', nan, '배출재검사(구변)', '구조변경검사',
       '종합(정밀) 1부재검사', '종합(정밀) 2부재검사', '종합(정밀)', '종합(경과:정밀)', '정밀검사',
       '특정재검사(배출)', '임시검사', '배출재검사', '관능재검사', '특정검사', '특정재검사(관능)',
       '정밀검사(경과)', '특정검사(경과)', 'A21R03', 'A21R05', 'A21R06', '관능재검사(임시)',
       'A21E03'], dtype=object)

## 검사방법 코드 변환

In [35]:
# 14.9s
mth_dict = {
    'A18A':'무부하검사(TSI)', 
    'A18B':'무부하검사(급가속)', 
    'A18C':'무부하검사(정지가동)',
    'A2301':'무부하검사(급가속)', 
    'A2302':'무부하검사(정지가동)',
    'A2303':'무부하검사(TSI)', 
    'A2304':'부하검사(LUG DOWN)', 
    'A2305':'부하검사(ASM-Idling)', 
    'A2306':'부하검사(KD-147)', 
    'A2307':'부하검사(ASM2525)', 
}
insm['검사방법'] = insm['검사방법'].replace(mth_dict)
insm['검사방법'].unique()

array(['부하검사(ASM-Idling)', '부하검사(KD-147)', '무부하검사(TSI)', '무부하검사(급가속)',
       '부하검사(LUG DOWN)', '무부하검사(정지가동)', '부하검사(ASM2525)'], dtype=object)

## 등록&제원&정기&정밀 병합

In [36]:
# 51.4s
csi = cs.merge(insm, on='차대번호', how='left')
csi.shape

(1390280, 28)

In [37]:
csi.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀'],
      dtype='object')

In [38]:
csi.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,검사종류,검사판정,무부하매연측정치1,무부하매연측정치2,무부하매연측정치3,무부하매연판정1,검사일자,주행거리,무부하매연허용치1,정기정밀
0,KMFZSX7JABU684137,4374025000,4,NaN,N,A0810006209153110,2011,화물,개인용,20220923.00,...,정기검사,Y,16.00,NaN,NaN,Y,20230623.00,223409.00,20.00,정기
1,KNCSE06428K344169,4374025000,4,NaN,N,A0110003113533107,2008,화물,개인용,20220923.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KMCGK17HP9C112030,4518010400,4,7MY-HD-24-28,N,A0810005008143209,2009,화물,개인용,20220407.00,...,정기(경과)검사,Y,5.00,NaN,NaN,Y,20230523.00,79425.00,20.00,정기
3,KNCSE01429K388374,2771026522,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210909.00,...,2부재검사(종합),Y,14.00,NaN,NaN,Y,20220810.00,50589.00,15.00,정밀
4,KNFSE0A439K387582,3020011900,4,7MY-KM-14-65,N,A0110003116073108,2009,화물,개인용,20210902.00,...,종합검사,Y,0.00,NaN,NaN,Y,20220831.00,151328.00,15.00,정밀


## 등록&제원&정기&정밀&법정동코드 병합

In [39]:
csi.shape

(1390280, 28)

In [40]:
csi['법정동코드'].isnull().sum()

0

In [41]:
csi['법정동코드'] = csi['법정동코드'].astype('str')
csi['법정동코드'] = csi['법정동코드'].str[:5] + '00000'
csi['법정동코드'] = pd.to_numeric(csi['법정동코드'])

In [42]:
df = csi.merge(coder, on='법정동코드', how='left')

In [43]:
df['시도'].isnull().sum()

0

In [44]:
df.shape

(1390280, 30)

In [45]:
df['시도'].isnull().sum()

0

In [46]:
df.loc[df['시도'].isnull(), '법정동코드']

Series([], Name: 법정동코드, dtype: int64)

In [47]:
# 주소 수정
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동

In [48]:
df['시도'].isnull().sum()

0

## rh 법정동코드 참고하여 법정동코드 수정

In [49]:
rdf = df.copy()

In [50]:
rdf['법정동코드'] = rdf['법정동코드'].astype('str')
rdf['법정동코드_mod'] = rdf['법정동코드'].str[:5] + '00000'
rdf['법정동코드_mod'] = pd.to_numeric(rdf['법정동코드_mod'])
rdf['법정동코드_mod'].head()

0    4374000000
1    4374000000
2    4518000000
3    2771000000
4    3020000000
Name: 법정동코드_mod, dtype: int64

In [51]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

{2817000000,
 3611000000,
 4119500000,
 4119700000,
 4119900000,
 4173000000,
 4371000000}

In [52]:
# 2817000000 인천광역시 남구 -> 인천광역시 미추홀구 2817700000
# 4119500000 경기도 부천시 원미구 -> 경기도 부천시 4119000000
# 4119700000 경기도 부천시 소사구 -> 경기도 부천시 4119000000
# 4119900000 경기도 부천시 오정구 -> 경기도 부천시 4119000000
# 4173000000 경기도 여주군 -> 경기도 여주시 4167000000
# 4371000000 충청북도 청원군 -> 충청북도 충주시 4311000000

In [53]:
rdf.loc[rdf['법정동코드_mod'] == 2817000000, '법정동코드_mod'] = 2817700000
rdf.loc[rdf['법정동코드_mod'] == 4119500000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119700000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119900000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4173000000, '법정동코드_mod'] = 4167000000
rdf.loc[rdf['법정동코드_mod'] == 4371000000, '법정동코드_mod'] = 4311000000

In [54]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

{3611000000}

In [55]:
rdf['법정동코드_mod'].isnull().sum()

0

In [56]:
df = rdf.copy()
df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_mod'],
      dtype='object')

## 4등급 result 파일 참고하여 DPF유무 수정

In [57]:
rdf = df.copy()

In [58]:
len(set(rdf['차대번호'].unique()) - set(rs['차대번호'].unique()))

2092

In [59]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879481
NaN     348422
유       261897
확인불가     12435
Name: count, dtype: int64

In [60]:
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rs.shape

(1502213, 2)

In [61]:
rdf1 = rdf.merge(rs, on='차대번호', how='left')

In [62]:
rdf1['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       806991
NaN     337658
유       234291
확인불가     11340
Name: count, dtype: int64

In [63]:
df = rdf1.copy()

In [64]:
df.shape

(1390280, 32)

## 연료 확인

In [65]:
df['연료'].value_counts(dropna=False)

연료
경유             1052802
휘발유             313038
LPG(액화석유가스)      24355
NaN                 49
기타연료                34
CNG(압축천연가스)          1
알코올                  1
Name: count, dtype: int64

## 연료 컬럼 추가

In [66]:
df.loc[df['연료'] == '경유', 'fuel'] = '경유'
df.loc[(df['연료'] == '휘발유') | (df['연료'] == 'LPG(액화석유가스)'), 'fuel'] = '휘발유_가스'

# 분석

## EG 분류

In [67]:
df['fuel'].value_counts(dropna=False)

fuel
경유        1052802
휘발유_가스     337393
NaN            85
Name: count, dtype: int64

In [68]:
grade_list = []
for f, y, cy, e in tqdm(df[['fuel', '제작일자', '차량연식', 'DPF유무_수정']].values):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 1390280/1390280 [00:02<00:00, 525249.22it/s]


1390280

In [69]:
df['Grade'] = grade_list

In [70]:
df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_mod', 'DPF유무_수정', 'fuel', 'Grade'],
      dtype='object')

# STD_BD_GRD4_CAR_CURSTT

## 4등급차만 추출

In [71]:
# 2.8s
df1 = df[df['배출가스등급'] == '4'].reset_index(drop=True)
df1.shape

(1390280, 34)

In [72]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230809'

In [73]:
df1['기준연월'] = '2022.12'
# df1['기준연월'] = today_date[:4] + '.' + today_date[4:6]

In [74]:
df1['테이블생성일자'] = today_date

In [75]:
df1.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_mod', 'DPF유무_수정', 'fuel', 'Grade', '기준연월', '테이블생성일자'],
      dtype='object')

In [76]:
df1['법정동코드_mod'] = df1['법정동코드_mod'].astype('str')
df1['법정동코드_mod'].head()

0    4374000000
1    4374000000
2    4518000000
3    2771000000
4    3020000000
Name: 법정동코드_mod, dtype: object

In [77]:
STD_BD_GRD4_CAR_CURSTT = df1[[
    '테이블생성일자', 
    '기준연월', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도',
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구',
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '배출가스등급', 
    'DPF유무_수정',
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '무부하매연측정치1', 
    '무부하매연판정1', 
    '무부하매연측정치2', 
    '무부하매연측정치3', 
    '법정동코드_mod', 
    'Grade', 
    ]]
STD_BD_GRD4_CAR_CURSTT.shape

(1390280, 26)

In [78]:
STD_BD_GRD4_CAR_CURSTT.head()

,테이블생성일자,기준연월,차대번호,제원관리번호,차종,용도,차량연식,차종유형,연료,법정동코드,...,DPF유무_수정,검사방법,검사종류,검사판정,무부하매연측정치1,무부하매연판정1,무부하매연측정치2,무부하매연측정치3,법정동코드_mod,Grade
0,20230809,2022.12,KMFZSX7JABU684137,A0810006209153110,화물,개인용,2011,소형,경유,4374000000,...,NaN,무부하검사(급가속),정기검사,Y,16.00,Y,NaN,NaN,4374000000,X
1,20230809,2022.12,KNCSE06428K344169,A0110003113533107,화물,개인용,2008,소형,경유,4374000000,...,무,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4374000000,C
2,20230809,2022.12,KMCGK17HP9C112030,A0810005008143209,화물,개인용,2009,중형,경유,4518000000,...,유,무부하검사(급가속),정기(경과)검사,Y,5.00,Y,NaN,NaN,4518000000,A
3,20230809,2022.12,KNCSE01429K388374,A0110003115353108,화물,개인용,2009,소형,경유,2771000000,...,무,부하검사(KD-147),2부재검사(종합),Y,14.00,Y,NaN,NaN,2771000000,C
4,20230809,2022.12,KNFSE0A439K387582,A0110003116073108,화물,개인용,2009,소형,경유,3020000000,...,무,부하검사(KD-147),종합검사,Y,0.00,Y,NaN,NaN,3020000000,C


In [79]:
ch_col_dict = {
                '테이블생성일자':'LOAD_DT',
                '기준연월':'CRTR_YM', 
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM', 
                '시군구':'SGG_NM', 
                '차명':'VHCNM', 
                '제작사명':'MNFCTR_NM', 
                '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
                '배출가스등급':'EXHST_GAS_GRD_CD_MOD',
                'DPF유무_수정':'DPF_MNTNG_YN', 
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1', 
                '무부하매연측정치2':'NOLOD_SMO_MEVLU2', 
                '무부하매연측정치3':'NOLOD_SMO_MEVLU3', 
                '법정동코드_mod':'STDG_CD_MOD',
                'Grade':'GRD4_MLSFC',

                '제작일자':'FBCTN_YMD', 
                '차종분류':'VHCTY_CL_CD',
                }

In [80]:
STD_BD_GRD4_CAR_CURSTT = STD_BD_GRD4_CAR_CURSTT.rename(columns=ch_col_dict)
STD_BD_GRD4_CAR_CURSTT.columns

Index(['LOAD_DT', 'CRTR_YM', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2',
       'YRIDNW', 'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM',
       'VHCNM', 'MNFCTR_NM', 'EXHST_GAS_CERT_NO_MOD', 'EXHST_GAS_GRD_CD_MOD',
       'DPF_MNTNG_YN', 'INSP_MTHD', 'INSP_KND', 'INSP_JGMT',
       'NOLOD_SMO_MEVLU1', 'NOLOD_SMO_JGMT_YN1', 'NOLOD_SMO_MEVLU2',
       'NOLOD_SMO_MEVLU3', 'STDG_CD_MOD', 'GRD4_MLSFC'],
      dtype='object')

In [81]:
STD_BD_GRD4_CAR_CURSTT.shape

(1390280, 26)

## [출력] STD_BD_GRD4_CAR_CURSTT

In [82]:
# expdf = STD_BD_GRD4_CAR_CURSTT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 9s
# we.import_from_pandas(expdf, table_nm)

In [83]:
# # 0s
# STD_BD_GRD4_CAR_CURSTT.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_CAR_CURSTT.csv'), index=False)

# STD_BD_GRD4_CAR_CURSTT_TOT

## 경유차만 추출

In [84]:
df2 = df1[df1['연료'] == '경유'].reset_index(drop=True)

## 차대번호 10자리 연식

In [85]:
df2['vin10'] = df2['차대번호'].str[9]

In [86]:
vin10_dict = {'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009, 'A':2010, 'B':2011, 'C':2012, 'D':2013, 'E':2014, 'F':2015, 'G':2016, 'H':2017}

In [87]:
df2['vin10_year'] = df2['vin10'].map(vin10_dict, na_action='ignore')

## 배인번호_수정 문자 타입으로 변경

In [88]:
df2['배출가스인증번호'] = df2['배출가스인증번호'].astype('str')

## 차대번호 17자리 샘플

In [89]:
df2y = df2.loc[df2['차대번호'].str.len() == 17].reset_index(drop=True)
df2n = df2.loc[df2['차대번호'].str.len() != 17].reset_index(drop=True)
df2y.shape, df2n.shape

((1052792, 38), (10, 38))

## 차대번호 연식과 연식 동일한 샘플

In [90]:
df3y = df2y.loc[df2y['vin10_year'] == df2y['차량연식']].reset_index(drop=True)
df3n = df2y.loc[df2y['vin10_year'] != df2y['차량연식']].reset_index(drop=True)
df3y.shape, df3n.shape

((1052708, 38), (84, 38))

# 배인번호별 분석

In [91]:
def flat_cols(df):
    df.columns = ['/'.join(x) for x in df.columns.to_flat_index()]
    return df

In [92]:
# about 31.3s
# 최적화 24m 51s -> 30.0s
total_g_df = pd.DataFrame()
groupby_col1 = ['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법', '검사판정']
groupby_col2 = ['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법']
for one in tqdm(df3y['배출가스인증번호'].unique()):
    # 배인번호별 df
    gas_df = df3y.loc[df3y['배출가스인증번호'] == str(one)].reset_index(drop=True)

    if gas_df.shape[0] != 0:
        # 제번별, 차형식별, 엔진형식별, 검사판정별 무부하매연측정치1 통계
        g = gas_df.groupby(groupby_col1).agg({'차대번호':'count', '무부하매연측정치1':['mean', 'min', 'max']}).pipe(flat_cols).round(2).reset_index()
        g = g.rename(columns={'배출가스인증번호':'배출가스인증번호', '차대번호/count':'대수', '무부하매연측정치1/mean':'mean', '무부하매연측정치1/min':'min', '무부하매연측정치1/max':'max'})
        # 하나의 배인번호에서 제번별 엔진형식별 비율 계산
        g['합격률(%)'] = round(g['대수'] / g.groupby(groupby_col2)['대수'].transform('sum') * 100, 2)
        # 종합 - 통계
        total_g_df = pd.concat([total_g_df, g], ignore_index=True)
    else:
        print(f'오류 배인번호 : {one}')
        pass

100%|██████████| 201/201 [00:16<00:00, 12.45it/s]


In [93]:
total_g_df.head()

,제작사명,배출가스인증번호,제원관리번호,자동차형식,엔진형식,검사종류,검사방법,검사판정,대수,mean,min,max,합격률(%)
0,(주) 골드밴,nan,B3M10009800003308,ECKH22-CAB,D6CB,종합검사,부하검사(LUG DOWN),Y,1,7.00,7.00,7.00,100.00
1,(주) 대진에스티씨,nan,1BE20002100003207,405 101,904LA,종합검사(경과),무부하검사(급가속),Y,1,0.00,0.00,0.00,100.00
2,(주) 대진에스티씨,nan,1BE20002500003307,405 221,906LA,정기검사,무부하검사(급가속),Y,1,1.00,1.00,1.00,100.00
3,(주) 대진에스티씨,nan,1BE20002600003307,405 122,906LA,정기검사,무부하검사(급가속),Y,1,0.00,0.00,0.00,100.00
4,(주) 대진에스티씨,nan,1BE20002800003307,405 221,906LA,종합검사,무부하검사(급가속),Y,1,1.00,1.00,1.00,100.00


# 분석

## 열화계수(SI) 지수 계산
- SI = 측정치 / 허용치

In [94]:
df3y.shape

(1052708, 38)

In [95]:
df3y.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_mod', 'DPF유무_수정', 'fuel', 'Grade', '기준연월', '테이블생성일자', 'vin10',
       'vin10_year'],
      dtype='object')

In [96]:
sidf = df3y[[
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '차량연식', 
    '차명', 
    '차종유형', 
    '제작사명', 
    '연료', 
    '법정동코드', 
    '배출가스인증번호', 
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '주행거리', 
    '무부하매연판정1', 
    '무부하매연허용치1', 
    '무부하매연측정치1'
    ]]
sidf.shape

(1052708, 17)

In [97]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [98]:
sidf['차령'] = current_yr - sidf['차량연식']
sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3260\4139794486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['차령'] = current_yr - sidf['차량연식']
C:\Users\kbjung\AppData\Local\Temp\ipykernel_3260\4139794486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']


In [99]:
sidf['차량연식'].isnull().sum()

0

In [100]:
sidf[['차령', 'SI']].isnull().sum()

차령        0
SI    46456
dtype: int64

In [101]:
sidf[['무부하매연측정치1', '무부하매연허용치1']].isnull().sum()

무부하매연측정치1    12653
무부하매연허용치1    12655
dtype: int64

In [102]:
sidf.head()

,차대번호,제원관리번호,차종,차량연식,차명,차종유형,제작사명,연료,법정동코드,배출가스인증번호,검사방법,검사종류,검사판정,주행거리,무부하매연판정1,무부하매연허용치1,무부하매연측정치1,차령,SI
0,KMFZSX7JABU684137,A0810006209153110,화물,2011,포터Ⅱ(PORTERⅡ),소형,현대자동차(주),경유,4374000000,nan,무부하검사(급가속),정기검사,Y,223409.00,Y,20.00,16.00,12,0.80
1,KNCSE06428K344169,A0110003113533107,화물,2008,봉고Ⅲ 1톤,소형,기아자동차(주),경유,4374000000,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN
2,KMCGK17HP9C112030,A0810005008143209,화물,2009,마이티군물자수송트럭,중형,현대자동차(주),경유,4518000000,7MY-HD-24-28,무부하검사(급가속),정기(경과)검사,Y,79425.00,Y,20.00,5.00,14,0.25
3,KNCSE01429K388374,A0110003115353108,화물,2009,봉고Ⅲ 1톤,소형,기아자동차(주),경유,2771000000,7MY-KM-14-65,부하검사(KD-147),2부재검사(종합),Y,50589.00,Y,15.00,14.00,14,0.93
4,KNFSE0A439K387582,A0110003116073108,화물,2009,봉고Ⅲ 냉동차,소형,기아자동차(주),경유,3020000000,7MY-KM-14-65,부하검사(KD-147),종합검사,Y,151328.00,Y,15.00,0.00,14,0.00


### 경유만 추출

In [103]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230809'

In [104]:
sidf['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3260\3010636050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['테이블생성일자'] = today_date


In [105]:
sidf1 = sidf[[
    '테이블생성일자', 
    '차대번호', 
    '제원관리번호', 
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '검사방법',
    '주행거리', 
    '차령',
    'SI', 
    ]]

In [106]:
chc_dict = {
    '테이블생성일자':'LOAD_DT',
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '차종':'VHCTY_CD', 
    '연식':'YRIDNW', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '차종유형':'VHCTY_TY', 
    '연료':'FUEL_CD',
    '법정동코드':'STDG_CD', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    '검사종류':'INSP_KND', 
    '검사판정':'INSP_JGMT', 
    '주행거리':'DRVNG_DSTNC',
    '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    '무부하매연허용치1':'NOLOD_SMO_PRMT_VAL1',
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '차령':'VHCAG',
    }

In [107]:
STD_BD_GRD4_SI = sidf1.rename(columns=chc_dict)
STD_BD_GRD4_SI.columns

Index(['LOAD_DT', 'VIN', 'MANG_MNG_NO', 'VHCNM', 'MNFCTR_NM',
       'EXHST_GAS_CERT_NO_MOD', 'INSP_MTHD', 'DRVNG_DSTNC', 'VHCAG', 'SI'],
      dtype='object')

In [108]:
STD_BD_GRD4_SI.head()

,LOAD_DT,VIN,MANG_MNG_NO,VHCNM,MNFCTR_NM,EXHST_GAS_CERT_NO_MOD,INSP_MTHD,DRVNG_DSTNC,VHCAG,SI
0,20230809,KMFZSX7JABU684137,A0810006209153110,포터Ⅱ(PORTERⅡ),현대자동차(주),nan,무부하검사(급가속),223409.00,12,0.80
1,20230809,KNCSE06428K344169,A0110003113533107,봉고Ⅲ 1톤,기아자동차(주),nan,NaN,NaN,15,NaN
2,20230809,KMCGK17HP9C112030,A0810005008143209,마이티군물자수송트럭,현대자동차(주),7MY-HD-24-28,무부하검사(급가속),79425.00,14,0.25
3,20230809,KNCSE01429K388374,A0110003115353108,봉고Ⅲ 1톤,기아자동차(주),7MY-KM-14-65,부하검사(KD-147),50589.00,14,0.93
4,20230809,KNFSE0A439K387582,A0110003116073108,봉고Ⅲ 냉동차,기아자동차(주),7MY-KM-14-65,부하검사(KD-147),151328.00,14,0.00


In [109]:
STD_BD_GRD4_SI.shape

(1052708, 10)

### [출력] SI 지수 정보(STD_BD_GRD4_SI)

In [110]:
# expdf = STD_BD_GRD4_SI
# table_nm = 'STD_BD_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 6s
# we.import_from_pandas(expdf, table_nm)

In [111]:
# # 0s
# STD_BD_GRD4_SI.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_SI.csv'), index=False)

## 동일 배번에서 제번별 매연 boxplot

In [112]:
total_g_df1 = total_g_df.loc[(total_g_df['배출가스인증번호'] != '확인불가') | (total_g_df['배출가스인증번호'] != 'nan')]
total_g_df1.shape

(28301, 13)

In [113]:
sample01 = total_g_df1.loc[total_g_df1['대수'] > 100].reset_index(drop=True)
sample01.shape

(1507, 13)

### quantile

In [114]:
# 4m 11.2s
boxplot_df = pd.DataFrame()
quantile_df = pd.DataFrame()
for one in sample01['배출가스인증번호'].unique(): # 통계에서 100대 초과인 배인번호만 추출
    temp_one = df3y.loc[(df3y['배출가스인증번호'] == one) & (df3y['검사판정'] == 'Y')].reset_index(drop=True) # 해당 배인번호 중 검사판정이 'Y'인 샘플만 추출
    if temp_one.shape[0] > 100: # 100대 초과면 진행
        for two in temp_one['검사종류'].unique(): # 검사종류별 샘플 추출
            temp_two = temp_one.loc[temp_one['검사종류'] == two].reset_index(drop=True)
            if temp_two.shape[0] > 100:
                for three in temp_two['검사방법'].unique(): # 검사방법별 샘플 추출
                    temp_three = temp_two.loc[temp_two['검사방법'] == three].reset_index(drop=True)
                    boxplot_df = pd.concat([boxplot_df, temp_three], ignore_index=True) # 해당 샘플만 추출하여 쌓기
                    if temp_three.shape[0] > 100:
                        xticks_list= []
                        data_list = []
                        for four in temp_three['제원관리번호'].unique():
                            temp_four = temp_three.loc[temp_three['제원관리번호'] == four].reset_index(drop=True).dropna(subset=['무부하매연측정치1'])
                            if temp_four.shape[0] > 100:
                                xticks_list.append(four)
                                data_list.append(temp_four['무부하매연측정치1'])
                                temp_four['q1'] = temp_four['무부하매연측정치1'].describe()['25%']
                                temp_four['q2'] = temp_four['무부하매연측정치1'].describe()['50%']
                                temp_four['q3'] = temp_four['무부하매연측정치1'].describe()['75%']
                                temp_four['차량대수'] = temp_four.shape[0]
                                quantile_df = pd.concat([quantile_df, temp_four], ignore_index=True) # 제번별 4분위 값 df형태로 저장

In [115]:
quantile_df.shape

(796161, 42)

In [116]:
quantile_df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_mod', 'DPF유무_수정', 'fuel', 'Grade', '기준연월', '테이블생성일자', 'vin10',
       'vin10_year', 'q1', 'q2', 'q3', '차량대수'],
      dtype='object')

In [174]:
quantile_df.drop_duplicates(['배출가스인증번호', '제원관리번호']).shape

(708, 42)

In [117]:
quantile_df1 = quantile_df.drop_duplicates(['배출가스인증번호', '제원관리번호', '검사방법', '검사종류'])
quantile_df1.shape

(1402, 42)

In [118]:
quantile_df1.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,fuel,Grade,기준연월,테이블생성일자,vin10,vin10_year,q1,q2,q3,차량대수
0,VF30U9HZHBS006438,1129000000,4,nan,N,00620000900021210,2011,승용,개인용,20110328.00,...,경유,A,2022.12,20230809,B,2011.00,0.00,5.00,10.00,184
184,VF34C9HZHAS010877,2771000000,4,nan,N,00620000800081209,2010,승용,개인용,20100818.00,...,경유,A,2022.12,20230809,A,2010.00,1.00,4.00,9.50,159
343,VF34HRHRJ9S003363,4161000000,4,nan,N,00620000800021208,2009,승용,개인용,20100623.00,...,경유,A,2022.12,20230809,9,2009.00,3.00,6.00,9.00,109
452,VF33HRHRJ8S007818,2617000000,4,nan,N,00620000400221207,2008,승용,개인용,20100405.00,...,경유,A,2022.12,20230809,8,2008.00,2.00,6.00,9.25,112
564,VF36DRHRJ8L004972,4157000000,4,nan,N,00620000500251207,2008,승용,개인용,20091102.00,...,경유,A,2022.12,20230809,8,2008.00,1.00,4.00,9.00,160


In [119]:
quantile_df1.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_mod', 'DPF유무_수정', 'fuel', 'Grade', '기준연월', '테이블생성일자', 'vin10',
       'vin10_year', 'q1', 'q2', 'q3', '차량대수'],
      dtype='object')

In [120]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230809'

In [121]:
quantile_df1['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3260\752261507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quantile_df1['테이블생성일자'] = today_date


In [122]:
STD_BD_GRD4_CAR_CURSTT_TOT = quantile_df1[[
    '테이블생성일자',
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3',
    '차량대수',
    ]]
STD_BD_GRD4_CAR_CURSTT_TOT.shape

(1402, 10)

In [123]:
chc_dict = {
    '테이블생성일자':'LOAD_DT', 
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '차종':'VHCTY_CD', 
    '용도':'PURPS_CD2', 
    '차종유형':'CHCTY_TY', 
    '법정동코드':'STDG_CD', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    '검사종류':'INSP_KND', 
    '검사판정':'INSP_JGMT', 
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    'q1':'LOWR_QRT',
    'q2':'MID_QRT',
    'q3':'UP_QRT',
    '차량대수':'VHCL_MKCNT',
    # '등급_수정':'EXHST_GAS_GRD_CD_MOD', 
    # 'DPF유무_수정':'DPF_MNTNG_YN', 
    # '시도명':'CTPV_NM', 
    # '시군구명':'SGG_NM', 
    # '차종분류':'VHCTY_CL_CD', 
    }

In [124]:
STD_BD_GRD4_CAR_CURSTT_TOT = STD_BD_GRD4_CAR_CURSTT_TOT.rename(columns=chc_dict)
STD_BD_GRD4_CAR_CURSTT_TOT.columns

Index(['LOAD_DT', 'VHCNM', 'MNFCTR_NM', 'MANG_MNG_NO', 'EXHST_GAS_CERT_NO_MOD',
       'INSP_MTHD', 'LOWR_QRT', 'MID_QRT', 'UP_QRT', 'VHCL_MKCNT'],
      dtype='object')

In [125]:
STD_BD_GRD4_CAR_CURSTT_TOT.shape

(1402, 10)

In [126]:
STD_BD_GRD4_CAR_CURSTT_TOT.head()

,LOAD_DT,VHCNM,MNFCTR_NM,MANG_MNG_NO,EXHST_GAS_CERT_NO_MOD,INSP_MTHD,LOWR_QRT,MID_QRT,UP_QRT,VHCL_MKCNT
0,20230809,3008 1.6 HDi MCP,한불모터스(주),00620000900021210,nan,부하검사(KD-147),0.00,5.00,10.00,184
184,20230809,308 1.6 HDi MCP,한불모터스(주),00620000800081209,nan,부하검사(KD-147),1.00,4.00,9.50,159
343,20230809,Nouvelle Peugeot 308SW HDi,한불모터스(주),00620000800021208,nan,부하검사(KD-147),3.00,6.00,9.00,109
452,20230809,푸조 NEW 307SW HDi,한불모터스(주),00620000400221207,nan,부하검사(KD-147),2.00,6.00,9.25,112
564,20230809,푸조 407 HDi FAP A6,한불모터스(주),00620000500251207,nan,부하검사(KD-147),1.00,4.00,9.00,160


In [127]:
set(STD_BD_GRD4_CAR_CURSTT_TOT['MNFCTR_NM'].unique()) - set(STD_BD_GRD4_CAR_CURSTT['MNFCTR_NM'].unique())

set()

In [128]:
set(STD_BD_GRD4_SI['MNFCTR_NM'].unique()) - set(STD_BD_GRD4_CAR_CURSTT['MNFCTR_NM'].unique())

set()

### [출력] STD_BD_GRD4_CAR_CURSTT_TOT

In [129]:
# expdf = STD_BD_GRD4_CAR_CURSTT_TOT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT_TOT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [130]:
# # 0s
# STD_BD_GRD4_CAR_CURSTT_TOT.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_CAR_CURSTT_TOT.csv'), index=False)

## 인증번호검토

In [131]:
cert_df1 = quantile_df1[[
    '테이블생성일자',
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3',
    '차량대수',
    ]].reset_index(drop=True)
cert_df1.shape

(1402, 10)

In [132]:
cert_df1.head()

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수
0,20230809,3008 1.6 HDi MCP,한불모터스(주),00620000900021210,nan,부하검사(KD-147),0.00,5.00,10.00,184
1,20230809,308 1.6 HDi MCP,한불모터스(주),00620000800081209,nan,부하검사(KD-147),1.00,4.00,9.50,159
2,20230809,Nouvelle Peugeot 308SW HDi,한불모터스(주),00620000800021208,nan,부하검사(KD-147),3.00,6.00,9.00,109
3,20230809,푸조 NEW 307SW HDi,한불모터스(주),00620000400221207,nan,부하검사(KD-147),2.00,6.00,9.25,112
4,20230809,푸조 407 HDi FAP A6,한불모터스(주),00620000500251207,nan,부하검사(KD-147),1.00,4.00,9.00,160


In [133]:
sample01.columns

Index(['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법', '검사판정',
       '대수', 'mean', 'min', 'max', '합격률(%)'],
      dtype='object')

In [134]:
cert_df1['q2_mean'] = cert_df1.groupby(['배출가스인증번호', '검사방법'])['q2'].transform('mean')
cert_df1

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean
0,20230809,3008 1.6 HDi MCP,한불모터스(주),00620000900021210,nan,부하검사(KD-147),0.00,5.00,10.00,184,3.08
1,20230809,308 1.6 HDi MCP,한불모터스(주),00620000800081209,nan,부하검사(KD-147),1.00,4.00,9.50,159,3.08
2,20230809,Nouvelle Peugeot 308SW HDi,한불모터스(주),00620000800021208,nan,부하검사(KD-147),3.00,6.00,9.00,109,3.08
3,20230809,푸조 NEW 307SW HDi,한불모터스(주),00620000400221207,nan,부하검사(KD-147),2.00,6.00,9.25,112,3.08
4,20230809,푸조 407 HDi FAP A6,한불모터스(주),00620000500251207,nan,부하검사(KD-147),1.00,4.00,9.00,160,3.08
...,...,...,...,...,...,...,...,...,...,...,...
1397,20230809,싼타페(SANTAFE),현대자동차(주),A0810006800761307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,1.00,105,0.00
1398,20230809,싼타페(SANTAFE),현대자동차(주),A0810006800781307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,3.00,223,0.00
1399,20230809,쎄라토,기아자동차(주),A0110003001211206,5MY-KM-24-64,부하검사(KD-147),4.00,7.00,14.00,205,8.00
1400,20230809,쎄라토,기아자동차(주),A0110003000981205,5MY-KM-24-64,부하검사(KD-147),5.00,9.00,14.75,262,8.00


In [135]:
cert_df1.loc[(cert_df1['q2'] >= cert_df1['q2_mean']*5) | (cert_df1['q2'] <= cert_df1['q2_mean']/5), '검토구분'] = '주의'
cert_df1['검토구분'] = cert_df1['검토구분'].fillna('양호')

In [136]:
cert_df1

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean,검토구분
0,20230809,3008 1.6 HDi MCP,한불모터스(주),00620000900021210,nan,부하검사(KD-147),0.00,5.00,10.00,184,3.08,양호
1,20230809,308 1.6 HDi MCP,한불모터스(주),00620000800081209,nan,부하검사(KD-147),1.00,4.00,9.50,159,3.08,양호
2,20230809,Nouvelle Peugeot 308SW HDi,한불모터스(주),00620000800021208,nan,부하검사(KD-147),3.00,6.00,9.00,109,3.08,양호
3,20230809,푸조 NEW 307SW HDi,한불모터스(주),00620000400221207,nan,부하검사(KD-147),2.00,6.00,9.25,112,3.08,양호
4,20230809,푸조 407 HDi FAP A6,한불모터스(주),00620000500251207,nan,부하검사(KD-147),1.00,4.00,9.00,160,3.08,양호
...,...,...,...,...,...,...,...,...,...,...,...,...
1397,20230809,싼타페(SANTAFE),현대자동차(주),A0810006800761307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,1.00,105,0.00,주의
1398,20230809,싼타페(SANTAFE),현대자동차(주),A0810006800781307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,3.00,223,0.00,주의
1399,20230809,쎄라토,기아자동차(주),A0110003001211206,5MY-KM-24-64,부하검사(KD-147),4.00,7.00,14.00,205,8.00,양호
1400,20230809,쎄라토,기아자동차(주),A0110003000981205,5MY-KM-24-64,부하검사(KD-147),5.00,9.00,14.75,262,8.00,양호


In [137]:
cert_df1.shape, cert_df1.drop_duplicates(['배출가스인증번호', '검사방법', '검토구분', '제원관리번호']).shape

((1402, 12), (1016, 12))

In [138]:
grp1 = cert_df1.groupby(['배출가스인증번호', '검사방법', '검토구분', '제작사명', '차명', '제원관리번호'])['차량대수'].sum().reset_index().sort_values('차량대수', ascending=False)
grp1 = grp1.drop_duplicates(['배출가스인증번호', '검사방법', '검토구분', '제원관리번호']).reset_index(drop=True)
grp1 = grp1.rename(columns={'제작사명':'대표제작사명', '차명':'대표차명'})
grp1 = grp1.drop('차량대수', axis=1)
grp1

,배출가스인증번호,검사방법,검토구분,대표제작사명,대표차명,제원관리번호
0,9MY-HD-14-21,부하검사(KD-147),양호,현대자동차(주),싼타페(SANTAFE),A0810006801591209
1,9MY-KM-14-22,부하검사(KD-147),양호,기아자동차(주),쏘렌토,A0110004400111209
2,6MY-KM-14-61,부하검사(KD-147),양호,기아자동차(주),스포티지,A0110003400441206
3,7MY-HD-14-46,부하검사(KD-147),양호,현대자동차(주),그랜드 스타렉스(GRAND STAREX),A0810007402962210
4,9MY-HD-14-21,부하검사(KD-147),양호,현대자동차(주),싼타페(SANTAFE),A0810006801661210
...,...,...,...,...,...,...
1011,7MY-HD-13-58,무부하검사(급가속),주의,현대자동차(주),싼타페(SANTAFE),A0810006801391308
1012,7MY-HD-24-30,부하검사(LUG DOWN),양호,현대자동차(주),메가트럭,A0810005211123309
1013,7MY-HD-14-68,무부하검사(급가속),주의,현대자동차(주),포터Ⅱ(PORTERⅡ),A0810006209453110
1014,nan,부하검사(KD-147),주의,에프씨에이코리아(주),300C,00220004900001306


In [139]:
cg1 = cert_df1.merge(grp1, on=['배출가스인증번호', '검사방법', '검토구분', '제원관리번호'], how='left')
cg1.shape

(1402, 14)

In [140]:
cg1

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean,검토구분,대표제작사명,대표차명
0,20230809,3008 1.6 HDi MCP,한불모터스(주),00620000900021210,nan,부하검사(KD-147),0.00,5.00,10.00,184,3.08,양호,한불모터스(주),3008 1.6 HDi MCP
1,20230809,308 1.6 HDi MCP,한불모터스(주),00620000800081209,nan,부하검사(KD-147),1.00,4.00,9.50,159,3.08,양호,한불모터스(주),308 1.6 HDi MCP
2,20230809,Nouvelle Peugeot 308SW HDi,한불모터스(주),00620000800021208,nan,부하검사(KD-147),3.00,6.00,9.00,109,3.08,양호,한불모터스(주),Nouvelle Peugeot 308SW HDi
3,20230809,푸조 NEW 307SW HDi,한불모터스(주),00620000400221207,nan,부하검사(KD-147),2.00,6.00,9.25,112,3.08,양호,한불모터스(주),푸조 NEW 307SW HDi
4,20230809,푸조 407 HDi FAP A6,한불모터스(주),00620000500251207,nan,부하검사(KD-147),1.00,4.00,9.00,160,3.08,양호,한불모터스(주),푸조 407 HDi FAP A6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,20230809,싼타페(SANTAFE),현대자동차(주),A0810006800761307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,1.00,105,0.00,주의,현대자동차(주),싼타페(SANTAFE)
1398,20230809,싼타페(SANTAFE),현대자동차(주),A0810006800781307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,3.00,223,0.00,주의,현대자동차(주),싼타페(SANTAFE)
1399,20230809,쎄라토,기아자동차(주),A0110003001211206,5MY-KM-24-64,부하검사(KD-147),4.00,7.00,14.00,205,8.00,양호,기아자동차(주),쎄라토
1400,20230809,쎄라토,기아자동차(주),A0110003000981205,5MY-KM-24-64,부하검사(KD-147),5.00,9.00,14.75,262,8.00,양호,기아자동차(주),쎄라토


In [164]:
temp = cg1.copy()

In [167]:
temp['dup'] = cg1['제원관리번호'].duplicated()

In [168]:
temp.loc[temp['dup'] == True].head()

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean,검토구분,대표제작사명,대표차명,dup
34,20230809,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,부하검사(KD-147),1.00,3.00,7.00,374,3.00,양호,현대자동차(주),이-카운티,True
35,20230809,이-마이티,현대자동차(주),A0810005008583209,7MY-HD-24-28,부하검사(KD-147),1.00,4.00,7.00,194,3.00,양호,현대자동차(주),이-마이티,True
36,20230809,이-마이티,현대자동차(주),A0810005008603209,7MY-HD-24-28,부하검사(KD-147),1.00,4.00,8.00,150,3.00,양호,현대자동차(주),이-마이티,True
37,20230809,이-카운티,현대자동차(주),A0810005602832208,7MY-HD-24-28,부하검사(KD-147),0.00,2.00,6.00,218,3.00,양호,현대자동차(주),이-카운티,True
38,20230809,이-카운티,현대자동차(주),A0810005602312207,7MY-HD-24-28,부하검사(KD-147),0.00,2.00,5.00,152,3.00,양호,현대자동차(주),이-카운티,True


In [169]:
temp.loc[temp['제원관리번호'] == 'A0810005603012209']

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean,검토구분,대표제작사명,대표차명,dup
15,20230809,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,8.00,11.00,327,6.56,양호,현대자동차(주),이-카운티,False
34,20230809,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,부하검사(KD-147),1.00,3.00,7.00,374,3.00,양호,현대자동차(주),이-카운티,True
39,20230809,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,무부하검사(급가속),1.00,3.00,8.00,247,2.75,양호,현대자동차(주),이-카운티,True
45,20230809,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,부하검사(LUG DOWN),2.00,7.00,11.00,130,6.56,양호,현대자동차(주),이-카운티,True


In [171]:
temp2 = temp.drop_duplicates('제원관리번호')
temp2.shape

(708, 15)

In [172]:
temp2['검토구분'].value_counts(dropna=False)

검토구분
양호    606
주의    102
Name: count, dtype: int64

In [161]:
cg1[['검토구분', '대표제작사명', '대표차명']].isnull().sum()

검토구분      0
대표제작사명    0
대표차명      0
dtype: int64

In [142]:
cg1.loc[cg1['제작사명'] != cg1['대표제작사명']]

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean,검토구분,대표제작사명,대표차명


In [143]:
cg1.loc[cg1['차명'] != cg1['대표차명']]

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean,검토구분,대표제작사명,대표차명


In [144]:
cg2 = cg1.drop_duplicates(['배출가스인증번호', '검사방법', '검토구분', '대표제작사명', '대표차명', '제원관리번호']).reset_index(drop=True)
cg2.shape

(1016, 14)

In [173]:
cg2['검토구분'].value_counts(dropna=False)

검토구분
양호    853
주의    163
Name: count, dtype: int64

In [145]:
list(cg2.columns)

['테이블생성일자',
 '차명',
 '제작사명',
 '제원관리번호',
 '배출가스인증번호',
 '검사방법',
 'q1',
 'q2',
 'q3',
 '차량대수',
 'q2_mean',
 '검토구분',
 '대표제작사명',
 '대표차명']

In [146]:
STD_BD_DAT_GRD4_CERT_NO_RVW = cg2[[
    '배출가스인증번호',
    '검사방법',
    '검토구분',
    '대표제작사명',
    '대표차명',
    '제원관리번호',
    'q1',
    'q2',
    'q3',
    '테이블생성일자',
]]

In [147]:
cdict = {
    '배출가스인증번호':'EXHST_GAS_CERT_NO',
    '검사방법':'INSP_MTHD',
    '검토구분':'RVW_SE',
    '대표제작사명':'RPRS_MNFCTR_NM',
    '대표차명':'RPRS_VHCNM', 
    '제원관리번호':'MANG_MNG_NO',
    'q1':'LOWR_QRT',
    'q2':'MID_QRT',
    'q3':'UP_QRT',
    '테이블생성일자':'LOAD_DT',
}
STD_BD_DAT_GRD4_CERT_NO_RVW = STD_BD_DAT_GRD4_CERT_NO_RVW.rename(columns=cdict)
STD_BD_DAT_GRD4_CERT_NO_RVW.columns

Index(['EXHST_GAS_CERT_NO', 'INSP_MTHD', 'RVW_SE', 'RPRS_MNFCTR_NM',
       'RPRS_VHCNM', 'MANG_MNG_NO', 'LOWR_QRT', 'MID_QRT', 'UP_QRT',
       'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_GRD4_CERT_NO_RVW

In [148]:
# expdf = STD_BD_DAT_GRD4_CERT_NO_RVW
# table_nm = 'STD_BD_DAT_GRD4_CERT_NO_RVW'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [149]:
# # 0s
# STD_BD_DAT_GRD4_CERT_NO_RVW.to_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_CERT_NO_RVW.csv'), index=False)

## 열화도 테이블

In [150]:
sidf.columns

Index(['차대번호', '제원관리번호', '차종', '차량연식', '차명', '차종유형', '제작사명', '연료', '법정동코드',
       '배출가스인증번호', '검사방법', '검사종류', '검사판정', '주행거리', '무부하매연판정1', '무부하매연허용치1',
       '무부하매연측정치1', '차령', 'SI', '테이블생성일자'],
      dtype='object')

In [151]:
sidf['차량연식'].nsmallest(1)

1052705    1988
Name: 차량연식, dtype: int64

In [152]:
sidf.groupby(['배출가스인증번호', '검사방법']).agg({'차량연식':lambda x : x.nsmallest(1)}).reset_index()

,배출가스인증번호,검사방법,차량연식
0,4MY-AD-20,무부하검사(급가속),2006
1,4MY-DC-33,무부하검사(급가속),2007
2,4MY-DC-33,부하검사(LUG DOWN),2007
3,4MY-KM-06,무부하검사(급가속),2008
4,4MY-KM-06,부하검사(KD-147),2008
...,...,...,...
380,C135,부하검사(KD-147),2008
381,nan,무부하검사(TSI),1996
382,nan,무부하검사(급가속),2002
383,nan,부하검사(KD-147),2005


In [153]:
grp2 = sidf.groupby(['배출가스인증번호', '검사방법']).agg({'제작사명':lambda x:x.value_counts().index[0], '차명':lambda x:x.value_counts().index[0], '차종':lambda x:x.value_counts().index[0], '연료':lambda x:x.value_counts().index[0], '차량연식':lambda x : x.nsmallest(1), 'SI':'mean'}).reset_index()
grp2 = grp2.rename(columns={'제작사명':'대표제작사명', '차명':'대표차명', '차종':'대표차종', '연료':'대표차연료', '차량연식':'최초연식', 'SI':'열화도'})
grp2

,배출가스인증번호,검사방법,대표제작사명,대표차명,대표차종,대표차연료,최초연식,열화도
0,4MY-AD-20,무부하검사(급가속),아우디폭스바겐코리아(주),아우디 A6 3.0 TDI Quattro,승용,경유,2006,0.34
1,4MY-DC-33,무부하검사(급가속),타타대우상용차(주),대우11.5톤장축카고트럭(DPF),화물,경유,2007,0.40
2,4MY-DC-33,부하검사(LUG DOWN),타타대우상용차(주),대우11.5톤장축카고트럭(DPF),화물,경유,2007,0.32
3,4MY-KM-06,무부하검사(급가속),기아자동차(주),봉고Ⅲ 일반덤프,화물,경유,2008,0.40
4,4MY-KM-06,부하검사(KD-147),기아자동차(주),봉고Ⅲ 일반덤프,화물,경유,2008,0.36
...,...,...,...,...,...,...,...,...
380,C135,부하검사(KD-147),쌍용자동차(주),액티언,승용,경유,2008,0.34
381,nan,무부하검사(TSI),쌍용자동차(주),코란도KJ,승용,경유,1996,0.00
382,nan,무부하검사(급가속),에프씨에이코리아(주),짚 랭글러,승용,경유,2002,0.15
383,nan,부하검사(KD-147),한불모터스(주),볼보 S80D5,승용,경유,2005,0.28


In [154]:
grp2['테이블생성일자'] = today_date

In [155]:
list(grp2.columns)

['배출가스인증번호',
 '검사방법',
 '대표제작사명',
 '대표차명',
 '대표차종',
 '대표차연료',
 '최초연식',
 '열화도',
 '테이블생성일자']

In [156]:
cdict = {
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '검사방법':'INSP_MTHD', 
    '대표제작사명':'RPRS_MNFCTR_NM', 
    '대표차명':'RPRS_VHCNM', 
    '대표차종':'RPRS_VHCTY_CD', 
    '대표차연료':'RPRS_FUEL', 
    '최초연식':'FRST_YRIDNW', 
    '열화도':'SI', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD4_SI = grp2.rename(columns=cdict)
STD_BD_DAT_GRD4_SI.columns

Index(['EXHST_GAS_CERT_NO', 'INSP_MTHD', 'RPRS_MNFCTR_NM', 'RPRS_VHCNM',
       'RPRS_VHCTY_CD', 'RPRS_FUEL', 'FRST_YRIDNW', 'SI', 'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_GRD4_SI

In [157]:
# expdf = STD_BD_DAT_GRD4_SI
# table_nm = 'STD_BD_DAT_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [158]:
# # 0s
# STD_BD_DAT_GRD4_SI.to_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_SI.csv'), index=False)

# code end